# Aggregration of Experiment Results

This notebook pulls results from the `output` folder and aggregates them in a large table that can than be formated, pivoted etc.

In [1]:
from pathlib import Path

OUTPUT_PATH = Path("/work/smt4/thulke/vieweg/SER/Code/sisyphus/output")

## Metrics

In [5]:
import pandas as pd
import glob

dfs = []
for path in map(Path, glob.glob(f"{OUTPUT_PATH}/**/metrics.csv", recursive=True)):
    if "pp" in path.parts:
        continue

    print(path)
    metrics_df = pd.read_csv(path)
    metrics_df["experiment"] = path.parts[-3]
    metrics_df["dataset"] = path.parts[-4]
    dfs.append(metrics_df)

metrics = pd.concat(dfs, axis=0, ignore_index=True)
metrics.head(20)

/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_finetune_facebook_wav2vec2-base-960h/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-53-english/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_featureextract_facebook_wav2vec2-base-960h/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/yuan/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_featureextract_jonatasgrosman_wav2vec2-large-xlsr-53-english/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/iem/lj_featureextract_facebook_wav2vec2-base-960h/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/iem/lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-53-english/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/iem/lj_finetune_facebook_wav2vec2-base-960h/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/i

,split,metric,value,experiment,dataset
0,train,eer,0.866667,lj_finetune_facebook_wav2vec2-base-960h,rav
1,val,eer,0.866667,lj_finetune_facebook_wav2vec2-base-960h,rav
2,test,eer,0.866667,lj_finetune_facebook_wav2vec2-base-960h,rav
3,train,beer_<,1.000000,lj_finetune_facebook_wav2vec2-base-960h,rav
4,train,beer_U,1.000000,lj_finetune_facebook_wav2vec2-base-960h,rav
5,train,beer_N,1.000000,lj_finetune_facebook_wav2vec2-base-960h,rav
6,train,beer_F,0.000000,lj_finetune_facebook_wav2vec2-base-960h,rav
7,train,beer_I,0.000000,lj_finetune_facebook_wav2vec2-base-960h,rav
8,train,beer_S,0.000000,lj_finetune_facebook_wav2vec2-base-960h,rav
9,train,beer_H,0.000000,lj_finetune_facebook_wav2vec2-base-960h,rav


In [3]:
from IPython.display import Markdown

for split in metrics.split.unique():
    # global metrics
    global_only_metrics = metrics.loc[
        ~metrics.metric.str.contains("_") ^ metrics.metric.str.contains("total")
    ]
    global_only_metrics = global_only_metrics.assign(
        metric=global_only_metrics.metric.apply(lambda s: s.split("_")[0])
    )

    global_pivot_metrics = (
        global_only_metrics.query("split == @split").pivot(
            index="experiment", columns=["dataset", "metric"], values="value"
        )
        * 100
    )

    if split == "test":
        global_pivot_metrics.loc["lj_finetune_lit", ("rav", "eer")] = 18.01
        global_pivot_metrics.loc["lj_featureextract_lit", ("rav", "eer")] = 43.47
        global_pivot_metrics.loc["yuan_lit", ("iem", "beer")] = 100 - 75.6
        global_pivot_metrics.loc["yuan_lit", ("iem", "eer")] = 100 - 75.5

    # beer per emo
    per_emo_metrics = metrics.loc[
        metrics.metric.str.contains("_") & ~metrics.metric.str.contains("total")
    ]
    per_emo_metrics = per_emo_metrics.assign(
        emotion=per_emo_metrics.metric.apply(lambda s: s.split("_")[-1])
    )

    per_emo_pivot_metrics = (
        per_emo_metrics.query("split == @split").pivot(
            index="experiment", columns=["dataset", "emotion"], values="value"
        )
        * 100
    )

    # print
    display(Markdown(f"# {split}"))
    display(global_pivot_metrics.round(2).fillna("--"))
    display(per_emo_pivot_metrics.round(2).fillna("--"))

# train

dataset                                               rav           iem      
metric                                                eer   beer    eer  beer
experiment                                                                   
lj_featureextract_facebook_wav2vec2-base-960h       40.00   2.54  23.65  1.54
lj_featureextract_jonatasgrosman_wav2vec2-large...  36.30   2.31  30.66  1.74
lj_finetune_facebook_wav2vec2-base-960h             86.67   6.00  67.95  3.92
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  17.04   1.78   48.8  3.36
yuan                                                87.50  87.33     --    --

dataset                                               rav                \
emotion                                                 <      U      N   
experiment                                                                
lj_featureextract_facebook_wav2vec2-base-960h        62.5  25.69  28.47   
lj_featureextract_jonatasgrosman_wav2vec2-large...  42.36  15.97  29.17   
lj_finetune_facebook_wav2vec2-base-960h             100.0  100.0  100.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...    0.0  51.39    0.0   
yuan                                                   --     --     --   

dataset                                                                        \
emotion                                                 F    I    S    H    E   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h         0.0  0.0  0.0  0.0  0.0   
lj_featureextract_jonatasgrosman_wav2vec2-large...  27.78  0.0  0.0  0.0  0.0   
lj_finetune_facebook_wav2vec2-base-960h               0.0  0.0  0.0  0.0  0.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  40.97  0.0  0.0  0.0  0.0   
yuan                                                   --   --   --   --   --   

dataset                                                       ...  iem       \
emotion                                               D       ...    x    >   
experiment                                                    ...             
lj_featureextract_facebook_wav2vec2-base-960h       0.0  0.0  ...  0.0  0.0   
lj_featureextract_jonatasgrosman_wav2vec2-large...  0.0  0.0  ...  0.0  0.0   
lj_finetune_facebook_wav2vec2-base-960h             0.0  0.0  ...  0.0  0.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  0.0  0.0  ...  0.0  0.0   
yuan                                                 --   --  ...   --   --   

dataset                                                                      \
emotion                                               Z    T    q    G    M   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h        --   --   --   --   --   
lj_featureextract_jonatasgrosman_wav2vec2-large...  0.0   --  0.0   --  0.0   
lj_finetune_facebook_wav2vec2-base-960h              --  0.0  0.0  0.0  0.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  0.0   --   --   --   --   
yuan                                                 --   --   --   --   --   

dataset                                                            
emotion                                               X    6    z  
experiment                                                         
lj_featureextract_facebook_wav2vec2-base-960h        --   --   --  
lj_featureextract_jonatasgrosman_wav2vec2-large...  0.0  0.0  0.0  
lj_finetune_facebook_wav2vec2-base-960h             0.0   --   --  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   --   --   --  
yuan                                                 --   --   --  

[5 rows x 123 columns]

# val

dataset                                               rav           iem      
metric                                                eer   beer    eer  beer
experiment                                                                   
lj_featureextract_facebook_wav2vec2-base-960h       46.67   4.08  38.94  2.46
lj_featureextract_jonatasgrosman_wav2vec2-large...  48.33   4.25   37.5  1.97
lj_finetune_facebook_wav2vec2-base-960h             86.67   6.00  63.46  3.92
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  28.33   2.16  49.04  3.32
yuan                                                93.33  93.75     --    --

dataset                                               rav                      \
emotion                                                 <      U      N     F   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h        87.5   12.5   87.5   0.0   
lj_featureextract_jonatasgrosman_wav2vec2-large...   87.5    0.0   75.0  50.0   
lj_finetune_facebook_wav2vec2-base-960h             100.0  100.0  100.0   0.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   12.5   75.0    0.0  25.0   
yuan                                                   --     --     --    --   

dataset                                                                      \
emotion                                               I    S    H    E    D   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h       0.0  0.0  0.0  0.0  0.0   
lj_featureextract_jonatasgrosman_wav2vec2-large...  0.0  0.0  0.0  0.0  0.0   
lj_finetune_facebook_wav2vec2-base-960h             0.0  0.0  0.0  0.0  0.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  0.0  0.0  0.0  0.0  0.0   
yuan                                                 --   --   --   --   --   

dataset                                                  ...  iem            \
emotion                                                  ...    x    >    Z   
experiment                                               ...                  
lj_featureextract_facebook_wav2vec2-base-960h       0.0  ...  0.0  0.0   --   
lj_featureextract_jonatasgrosman_wav2vec2-large...  0.0  ...  0.0  0.0  0.0   
lj_finetune_facebook_wav2vec2-base-960h             0.0  ...  0.0  0.0   --   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  0.0  ...  0.0  0.0  0.0   
yuan                                                 --  ...   --   --   --   

dataset                                                                      \
emotion                                               T    q    G    M    X   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h        --   --   --   --   --   
lj_featureextract_jonatasgrosman_wav2vec2-large...   --  0.0   --  0.0  0.0   
lj_finetune_facebook_wav2vec2-base-960h             0.0  0.0  0.0  0.0  0.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   --   --   --   --   --   
yuan                                                 --   --   --   --   --   

dataset                                                       
emotion                                               6    z  
experiment                                                    
lj_featureextract_facebook_wav2vec2-base-960h        --   --  
lj_featureextract_jonatasgrosman_wav2vec2-large...  0.0  0.0  
lj_finetune_facebook_wav2vec2-base-960h              --   --  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   --   --  
yuan                                                 --   --  

[5 rows x 123 columns]

# test

dataset                                               rav           iem      
metric                                                eer   beer    eer  beer
experiment                                                                   
lj_featureextract_facebook_wav2vec2-base-960h       56.33   3.42  48.04  2.71
lj_featureextract_jonatasgrosman_wav2vec2-large...  52.33    3.9  46.67  2.35
lj_finetune_facebook_wav2vec2-base-960h             86.67    6.0  72.75  3.92
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  32.67   2.26  49.41  3.19
yuan                                                 86.0  85.94     --    --
lj_finetune_lit                                     18.01     --     --    --
lj_featureextract_lit                               43.47     --     --    --
yuan_lit                                               --     --   24.5  24.4

dataset                                               rav                      \
emotion                                                 <      U      N     F   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h        75.0   40.0   42.5   0.0   
lj_featureextract_jonatasgrosman_wav2vec2-large...   72.5   30.0   37.5  55.0   
lj_finetune_facebook_wav2vec2-base-960h             100.0  100.0  100.0   0.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   45.0   42.5    0.0  30.0   
yuan                                                   --     --     --    --   

dataset                                                                      \
emotion                                               I    S    H    E    D   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h       0.0  0.0  0.0  0.0  0.0   
lj_featureextract_jonatasgrosman_wav2vec2-large...  0.0  0.0  0.0  0.0  0.0   
lj_finetune_facebook_wav2vec2-base-960h             0.0  0.0  0.0  0.0  0.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  0.0  0.0  0.0  0.0  0.0   
yuan                                                 --   --   --   --   --   

dataset                                                  ...  iem            \
emotion                                                  ...    x    >    Z   
experiment                                               ...                  
lj_featureextract_facebook_wav2vec2-base-960h       0.0  ...  0.0  0.0   --   
lj_featureextract_jonatasgrosman_wav2vec2-large...  0.0  ...  0.0  0.0  0.0   
lj_finetune_facebook_wav2vec2-base-960h             0.0  ...  0.0  0.0   --   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  0.0  ...  0.0  0.0  0.0   
yuan                                                 --  ...   --   --   --   

dataset                                                                      \
emotion                                               T    q    G    M    X   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h        --   --   --   --   --   
lj_featureextract_jonatasgrosman_wav2vec2-large...   --  0.0   --  0.0  0.0   
lj_finetune_facebook_wav2vec2-base-960h             0.0  0.0  0.0  0.0  0.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   --   --   --   --   --   
yuan                                                 --   --   --   --   --   

dataset                                                       
emotion                                               6    z  
experiment                                                    
lj_featureextract_facebook_wav2vec2-base-960h        --   --  
lj_featureextract_jonatasgrosman_wav2vec2-large...  0.0  0.0  
lj_finetune_facebook_wav2vec2-base-960h              --   --  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   --   --  
yuan                                                 --   --  

[5 rows x 123 columns]